# ENUM_BANK의 주소를 bloomberg 에서 가져오는 Crawler
## 수정해야 하는 곳
### 3번셀에서 csv파일명 수정
### 5번셀에서 크롬드라이버의 경로
### 

## 필요한 라이브러리들 인젝션

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import json
import pandas as pd
import re
import time

## 셀레늄에 헤더정보 설정 및 헤더리스 설정

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')
chrome_options.add_argument('window-size=1920x1080')
chrome_options.add_argument("disable-gpu")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.108 Safari/537.36")
chrome_options.add_argument("lang=ko_KR") # 한국어!

## GIVEN
### bing에서 bloomberg company profile + 기업명 검색결과 주소중 첫번째 검색결과 링크를 블룸버그경로로 설정

In [3]:
# 크롤링할 csv 파일 만들어서 이곳에서 교체
df = pd.read_csv('Bangladesh-Bank.csv')
df_enum_bank_japan = df
target_list = df_enum_bank_japan.descr

In [4]:
df_enum_bank_japan

,Unnamed: 0,descr
0,NaN,Mutual Trust Bank
1,NaN,Agrani Bank
2,NaN,Al-Arafah Islami Bank
3,NaN,AB Bank
4,NaN,Bangladesh Bank
5,NaN,Bangladesh Commerce Bank
6,NaN,Bangladesh Krishi Bank
7,NaN,Bangladesh Samabaya Bank
8,NaN,Bangladesh Development Bank
9,NaN,Basic Bank


In [5]:
driver = webdriver.Chrome('/Users/arthur/pywork/chromedriver', options=chrome_options)
driver.get('https://transferwise.com/gb/swift-codes/banks?country=0&city=0&name=THE+TOHOKU+BANK')
soup = BeautifulSoup(driver.page_source,'html.parser')

In [6]:
tmp_addr = soup.select('div div div div div div ul li small span')
print(tmp_addr[1].text)
tmp_bank = soup.select('div div div div div div ul li div a')
print(tmp_bank[0].text)

3-1 Uchimaru Morioka 020 0023, 
The Tohoku Bank, Ltd


## WHEN

In [7]:
addr = []
bank = []
url = 'https://transferwise.com/gb/swift-codes/banks?country=313&city=0&name='
descr_list = df_enum_bank_japan.descr
for idx, df_data in enumerate(descr_list) :
    print(idx)
    tmp_str = url + df_data
    driver.get(tmp_str)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    tmp_bank = soup.select('div div div div div div ul li div a')
    tmp_addr = soup.select('div div div div div div ul li small span')
    bank.append(tmp_bank[0].text)
    addr.append(tmp_addr[1].text)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58


In [8]:
result = {'bank':bank, 'descr':df_enum_bank_japan.descr, 'addr':addr}
df_result = pd.DataFrame(result)
df_result.head()

,bank,descr,addr
0,Mutual Trust Bank Limited,Mutual Trust Bank,"37 Agrabad C/a Chittagong,"
1,Agrani Bank Limited,Agrani Bank,"Agrani Bank Bhavan 1000 Dhaka,"
2,Al-arafah Islami Bank Ltd.,Al-Arafah Islami Bank,"Rahman Mension 1000 Dhaka,"
3,Agrani Bank Limited,AB Bank,"Agrani Bank Bhavan 1000 Dhaka,"
4,Bangladesh Krishi Bank,Bangladesh Bank,"Krishi Bank Bhaban 1000 Dhaka,"


In [9]:
df_result.to_csv('tmp.csv', header=False, index=False)